In [9]:
import pandas as pd
import numpy as np

In [10]:
# Fungsi Keanggotaan Umum
def fungsi_segitiga(x, a, b, c):
    term1 = 0
    if b - a != 0: term1 = (x - a) / (b - a)
    term2 = 0
    if c - b != 0: term2 = (c - x) / (c - b)
    return max(0, min(term1, term2))


def fungsi_trapesium(x, a, b, c, d):
    term1 = 0
    if b - a != 0: term1 = (x - a) / (b - a)
    term2 = 0
    if d - c != 0: term2 = (d - x) / (d - c)
    return max(0, min(term1, 1, term2))

In [11]:
# Fuzifikasi untuk setiap variabel input
def fuzzify_age(age_val):
    return {
        'Muda': fungsi_trapesium(age_val, 0, 0, 35, 45),
        'ParuhBaya': fungsi_segitiga(age_val, 40, 50, 60),
        'Tua': fungsi_trapesium(age_val, 55, 65, 70, 75),
        'SangatTua': fungsi_trapesium(age_val, 70, 80, 100, 100)
    }

def fuzzify_glucose(glucose_val):
    return {
        'Normal': fungsi_trapesium(glucose_val, 50, 50, 90, 105),
        'Perbatasan': fungsi_segitiga(glucose_val, 100, 115, 130),
        'Tinggi': fungsi_trapesium(glucose_val, 120, 140, 272, 272)
    }

def fuzzify_bmi(bmi_val):
    return {
        'Kurus': fungsi_trapesium(bmi_val, 10, 10, 17, 18.5),
        'Normal': fungsi_trapesium(bmi_val, 17.5, 18.5, 24, 25.5),
        'Berlebih': fungsi_segitiga(bmi_val, 24.5, 27.5, 31),
        'Obesitas': fungsi_trapesium(bmi_val, 29, 32, 98, 100)
    }

In [12]:
# Output Linguistik dan Fungsi Keanggotaan Output
def fuzzify_output_risk(x):
    return {
        'SangatRendah': fungsi_trapesium(x, 0, 0, 15, 30),
        'Rendah': fungsi_segitiga(x, 25, 40, 55),
        'Sedang': fungsi_segitiga(x, 50, 65, 80),
        'Tinggi': fungsi_trapesium(x, 75, 90, 100, 100)
    }

In [13]:
# Aturan Fuzzy Mamdani
def apply_rules_mamdani(age_fuzzy, glucose_fuzzy, bmi_fuzzy, hypertension, heart_disease):
    rules = [
        # Aturan dasar
        ('SangatTua', 'Tinggi',     'Obesitas', 'Tinggi'),
        ('Tua',       'Tinggi',     'Obesitas', 'Tinggi'),
        ('SangatTua', 'Tinggi',     'Berlebih', 'Tinggi'),
        ('SangatTua', 'Perbatasan', 'Obesitas', 'Tinggi'),
        ('Tua',       'Tinggi',     'Berlebih', 'Sedang'),
        ('Tua',       'Perbatasan', 'Obesitas', 'Sedang'),
        ('ParuhBaya', 'Tinggi',     'Obesitas', 'Sedang'),
        ('SangatTua', 'Normal',     'Normal',   'Sedang'),
        ('Tua',       'Tinggi',     'Normal',   'Sedang'),
        ('Tua',       'Normal',     'Obesitas', 'Rendah'),
        ('ParuhBaya', 'Tinggi',     'Berlebih', 'Rendah'),
        ('Muda',      'Tinggi',     'Obesitas', 'Rendah'),
        ('Tua',       'Normal',     'Normal',   'Rendah'),
        ('ParuhBaya', 'Normal',     'Normal',   'SangatRendah'),
        ('Muda',      'Normal',     'Normal',   'SangatRendah'),
    ]

    # Logika peningkat risiko
    risk_upgrade_1 = {'SangatRendah': 'Rendah', 'Rendah': 'Sedang', 'Sedang': 'Tinggi', 'Tinggi': 'Tinggi'}
    risk_upgrade_2 = {'SangatRendah': 'Sedang', 'Rendah': 'Tinggi', 'Sedang': 'Tinggi', 'Tinggi': 'Tinggi'}

    output_membership = {label: [] for label in fuzzify_output_risk(0).keys()}

    for age_label, glucose_label, bmi_label, out_label_base in rules:
        w = min(age_fuzzy[age_label], glucose_fuzzy[glucose_label], bmi_fuzzy[bmi_label])
        
        final_out_label = out_label_base
        num_conditions = hypertension + heart_disease
        if num_conditions == 1:
            final_out_label = risk_upgrade_1[out_label_base]
        elif num_conditions == 2:
            final_out_label = risk_upgrade_2[out_label_base]
        
        output_membership[final_out_label].append(w)
    
    return {k: max(v) if v else 0 for k, v in output_membership.items()}


In [14]:
# Defuzzifikasi Mamdani
def defuzzify_mamdani(output_membership):
    x = np.linspace(0, 100, 101) 
    aggregated = np.zeros_like(x)
    for label, degree in output_membership.items():
        if degree > 0:
            mf = np.array([fuzzify_output_risk(xi)[label] for xi in x])
            aggregated = np.maximum(aggregated, np.minimum(degree, mf))
    if aggregated.sum() == 0:
        return 0
    return np.sum(x * aggregated) / np.sum(aggregated)

In [15]:
# Prediksi satu baris
def predict_stroke_risk_mamdani(input_data):
    age_fuzzy = fuzzify_age(input_data['age'])
    glucose_fuzzy = fuzzify_glucose(input_data['avg_glucose_level'])
    bmi_fuzzy = fuzzify_bmi(input_data['bmi'])
    output_membership = apply_rules_mamdani(
        age_fuzzy, glucose_fuzzy, bmi_fuzzy,
        input_data['hypertension'], input_data['heart_disease']
    )
    return defuzzify_mamdani(output_membership)

# Uji Evaluasi Model Mamdani

In [16]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Load dataset yang telah diproses
df = pd.read_csv('stroke_preprocessed_for_fuzzy.csv')

def predict_row(row):
    input_data = {
        'age': row['age'],
        'avg_glucose_level': row['avg_glucose_level'],
        'bmi': row['bmi'],
        'hypertension': row['hypertension'],
        'heart_disease': row['heart_disease']
    }
    return predict_stroke_risk_mamdani(input_data)

# Menghitung skor risiko stroke menggunakan Fuzzy Mamdani
df['mamdani_risk_score'] = df.apply(predict_row, axis=1)

# Menggunakan threshold 0.5 untuk klasifikasi
df['mamdani_pred'] = (df['mamdani_risk_score'] >= 0.5).astype(int)

# Mengambil label target dari dataset sebenarnya dan hasil prediksi
y_true = df['stroke']

# Mengambil thereshold optimal menggunakan F1-score
thresholds = np.arange(1, 100, 1) 
f1_scores = [f1_score(y_true, (df['mamdani_risk_score'] >= thresh).astype(int), zero_division=0) for thresh in thresholds]
optimal_idx = np.argmax(f1_scores)

# Menentukan threshold optimal
optimal_threshold = thresholds[optimal_idx]

y_pred_optimal = (df['mamdani_risk_score'] >= optimal_threshold).astype(int)

# Evaluasi model
acc = accuracy_score(y_true, y_pred_optimal)
f1 = f1_score(y_true, y_pred_optimal, zero_division=0)
precision = precision_score(y_true, y_pred_optimal, zero_division=0)
recall = recall_score(y_true, y_pred_optimal, zero_division=0)

print("=== Evaluasi Fuzzy Mamdani ===")
print(f"Akurasi   : {acc:.4f}")
print(f"F1-score  : {f1:.4f}")
print(f"Precision : {precision:.4f}")
print(f"Recall    : {recall:.4f}")

=== Evaluasi Fuzzy Mamdani ===
Akurasi   : 0.9381
F1-score  : 0.0186
Precision : 0.0411
Recall    : 0.0120
